In [10]:
import ipywidgets as widgets 
from IPython.display import display 
# from threading import Thread
import _thread

from queue import Queue

messages = Queue()
recording = Queue()

record_button = widgets.Button(
    description= "Record", 
    disable = False , 
    button_style = "success",
    icon = "microphone "
)

stop_button = widgets.Button(
    description= "Record", 
    disable = False , 
    button_style = "warning",
    icon = "stop"
)

output = widgets.Output()
def start_recording(data):
    messages.put(True)

    with output: 
        display("Starting ...")
        record = _thread(target=record_microphone)
        record.start()

        transcribe = _thread(target=speech_recognition,args = (output,))
        transcribe.start()
    pass 

def stop_recording(data):
    with output: 
        messages.get()
        display("Stopped.")
    pass 

record_button.on_click(start_recording)
stop_button.on_click(stop_recording)


display(record_button,stop_button,output)

Button(button_style='success', description='Record', icon='microphone ', style=ButtonStyle())

Button(button_style='warning', description='Record', icon='stop', style=ButtonStyle())

Output()

In [2]:
import pyaudio

p = pyaudio.PyAudio()
for i in range (p.get_device_count()):
    print(p.get_device_info_by_index(i))

p.terminate()

{'index': 0, 'structVersion': 2, 'name': 'sysdefault', 'hostApi': 0, 'maxInputChannels': 128, 'maxOutputChannels': 128, 'defaultLowInputLatency': 0.021333333333333333, 'defaultLowOutputLatency': 0.021333333333333333, 'defaultHighInputLatency': 0.021333333333333333, 'defaultHighOutputLatency': 0.021333333333333333, 'defaultSampleRate': 48000.0}
{'index': 1, 'structVersion': 2, 'name': 'front', 'hostApi': 0, 'maxInputChannels': 0, 'maxOutputChannels': 2, 'defaultLowInputLatency': -1.0, 'defaultLowOutputLatency': 0.005333333333333333, 'defaultHighInputLatency': -1.0, 'defaultHighOutputLatency': 0.032, 'defaultSampleRate': 48000.0}
{'index': 2, 'structVersion': 2, 'name': 'surround40', 'hostApi': 0, 'maxInputChannels': 0, 'maxOutputChannels': 2, 'defaultLowInputLatency': -1.0, 'defaultLowOutputLatency': 0.005333333333333333, 'defaultHighInputLatency': -1.0, 'defaultHighOutputLatency': 0.032, 'defaultSampleRate': 48000.0}
{'index': 3, 'structVersion': 2, 'name': 'surround51', 'hostApi': 0, 

ALSA lib conf.c:4028:(snd_config_hooks_call) Cannot open shared library libasound_module_conf_pulse.so (/home/jeo/miniconda3/envs/speech/lib/alsa-lib/libasound_module_conf_pulse.so: cannot open shared object file: No such file or directory)
ALSA lib control.c:1570:(snd_ctl_open_noupdate) Invalid CTL hw:0
ALSA lib pcm.c:2675:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2675:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2675:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_route.c:878:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:878:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:878:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:878:(find_matching_chmap) Found no matching channel map
ALSA lib dlmisc.c:339:(snd_dlobj_cache_get0) Cannot open shared library libasound_module_rate_samplerate.so (/home/jeo/miniconda3/envs/speech/lib/alsa-lib/libasound_mo

In [3]:
import sounddevice as sd 

print(sd.query_devices())
device_info = sd.query_devices(sd.default.device[0], 'input')
samplerate = int(device_info['default_samplerate'])

# display the default input device
print("===> Initial Default Device Number:{} Description: {}".format(sd.default.device[0], device_info))

  0 sysdefault, ALSA (128 in, 128 out)
  1 front, ALSA (0 in, 2 out)
  2 surround40, ALSA (0 in, 2 out)
  3 surround51, ALSA (0 in, 2 out)
  4 surround71, ALSA (0 in, 2 out)
  5 hdmi, ALSA (0 in, 2 out)
* 6 default, ALSA (128 in, 128 out)
  7 dmix, ALSA (0 in, 2 out)
===> Initial Default Device Number:6 Description: {'name': 'default', 'index': 6, 'hostapi': 0, 'max_input_channels': 128, 'max_output_channels': 128, 'default_low_input_latency': 0.021333333333333333, 'default_low_output_latency': 0.021333333333333333, 'default_high_input_latency': 0.021333333333333333, 'default_high_output_latency': 0.021333333333333333, 'default_samplerate': 48000.0}


In [4]:
device_info = sd.query_devices(sd.default.device[0], 'input')
samplerate = int(device_info['default_samplerate'])
device_info

{'name': 'default',
 'index': 6,
 'hostapi': 0,
 'max_input_channels': 128,
 'max_output_channels': 128,
 'default_low_input_latency': 0.021333333333333333,
 'default_low_output_latency': 0.021333333333333333,
 'default_high_input_latency': 0.021333333333333333,
 'default_high_output_latency': 0.021333333333333333,
 'default_samplerate': 48000.0}

In [5]:
print("===> Initial Default Device Number:{} Description: {}".format(sd.default.device[0], device_info))

===> Initial Default Device Number:6 Description: {'name': 'default', 'index': 6, 'hostapi': 0, 'max_input_channels': 128, 'max_output_channels': 128, 'default_low_input_latency': 0.021333333333333333, 'default_low_output_latency': 0.021333333333333333, 'default_high_input_latency': 0.021333333333333333, 'default_high_output_latency': 0.021333333333333333, 'default_samplerate': 48000.0}


In [6]:
device_info.get("index")

6

In [7]:
CHANNELS = 1 
FRAME_RATE = device_info.get('default_samplerate') 
RECORD_SECONDS = 20 
AUDIO_FORMAT = 'int16'
SAMPLE_SIZE = 2

def record_microphone(chunk = 1024):
    stream = sd.RawInputStream(dtype= AUDIO_FORMAT,
                        channels= CHANNELS,
                        samplerate= FRAME_RATE,
                        input = True, 
                        prime_output_buffers_using_stream_callback=chunk)
    frames = []

    while not messages.empty():
        data = stream.read(chunk)
        frames.append(data)

        if len(frame) >= (FRAME_RATE * RECORD_SECONDS) / chunk:
            recording.put(frames.copy())
            frame = []
        
    stream.stop_stream()
    stream.close()
    sd._terminate()


In [ ]:
%pip install torch 

In [8]:
import subprocess 
import json 
from vosk import Model,KaldiRecognizer 

model = Model(model_name = "vosk-model-en-us-0.22")
rec = KaldiRecognizer(model, FRAME_RATE)
rec.SetWords(True)

def speech_recognition(output):
    while not messages.empty():
        frames = recording.get()

        rec.AcceptWaveform(b''.join(frames))
        results = rec.Result()
        text = json.loads(results)["text"]



LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=13 max-active=7000 lattice-beam=6
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:11:12:13:14:15
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 0 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 0 orphan components.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from /home/jeo/.cache/vosk/vosk-model-en-us-0.22/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:279) Loading HCLG from /home/jeo/.cache/vosk/vosk-model-en-us-0.22/graph/HCLG.fst
LOG (VoskAPI:ReadDataFiles():model.cc:297) Loading words from /home/jeo/.cache/vosk/vosk-model-en-us-0.22/graph/words.txt
LOG (VoskAPI:ReadDataFiles():model.cc:308) Loading winfo /home/jeo/.cache/vosk/vosk-model-en-us-0.22/gra